In [1]:
import numpy as np
import torch
import re
import jieba

In [2]:
stop_word_path = "/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/NLP数据集合/停词库/stop_word_for_chinese.txt"

def del_element(strings,symbles):
    srcrep = {i:'' for i in symbles }
    rep = dict((re.escape(k), v) for k, v in srcrep.items())
    pattern = re.compile("|".join(rep.keys()))
    return pattern.sub(lambda m: rep[re.escape(m.group(0))], strings)
    
def filter_stop_word(strings,stop_word=np.loadtxt(stop_word_path,dtype=str)):
    return del_element(strings,stop_word)

In [3]:
def read_txt(path):
    return open(path,'r').readlines()

In [4]:
corpus = "/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/NLP数据集合/豆瓣电影数据集(2019.3)/豆瓣电影简介.txt"

In [5]:
def split_word(corpus,n):
    return np.array(list(jieba.cut(filter_stop_word(filter_stop_word(read_txt(corpus)[n]),'\n '))))

In [6]:
def just_number(string, resymbol=""):
    sub_str = re.sub(u"([^\u0030-\u0039])", resymbol, string)
    return sub_str

def sigmoid(z):
    return 1/(1+np.exp(-z))

def filter_dict(words,numbers):
    word_dict ,expr_sum = dict(),0
    for word,num in zip(words,numbers):
        try:
            number = float(just_number(num))
            expr = {word:number}  
        except Exception:
            number = 0
        finally:
            expr_sum += number
            word_dict.update(expr)
    prob = {word:word_dict[word]/expr_sum for word in word_dict}
    return prob

def location_dict(dir_path):
    init_dict = np.zeros((2,2))
    for path in os.listdir(dir_path):
        try:
            file_path = "{}{}".format(dir_path,path)
            file_of_one = np.loadtxt(file_path,delimiter='\t',dtype=str)
            print("ok:",path)
        except Exception as error:
            file_of_one = np.array([line.replace("\n","").split("\t") for line in open(file_path,'r').readlines()])
            print("error:",path,error)
        finally :
            init_dict = np.r_[init_dict,file_of_one]
    words,numbers = init_dict[2:,0],init_dict[2:,1]
    return filter_dict(words,numbers)



dir_path = "/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/NLP数据集合/词库/chinese/"
words_dict = location_dict(dir_path)

ok: THUOCL_food.txt
ok: THUOCL_chengyu.txt
ok: THUOCL_caijing.txt
ok: THUOCL_law.txt
ok: THUOCL_medical.txt
ok: THUOCL_poem.txt
ok: THUOCL_animal.txt
ok: THUOCL_lishimingren.txt
error: THUOCL_it.txt Wrong number of columns at line 89
ok: THUOCL_diming.txt
ok: THUOCL_car.txt


In [7]:
corpus = "/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/NLP数据集合/豆瓣电影数据集(2019.3)/豆瓣电影简介.txt"
src1,src2 = split_word(corpus,1),split_word(corpus,5)

Building prefix dict from /usr/local/lib/python3.7/site-packages/jieba/dict.txt ...
Loading model from cache /var/folders/sl/q8x6_03132dfk7rktf00yh880000gn/T/jieba.cache
Loading model cost 1.4183447360992432 seconds.
Prefix dict has been built succesfully.


In [8]:
src1,src2

(array(['里昂', '雷诺', '饰名', '孤', '职业杀手', '受', '雇佣', '天', '邻居家', '姑娘', '马蒂',
        '达纳', '塔丽', '波特曼', '饰', '敲开', '房门', '求里', '暂避', '杀身', '祸', '邻居家',
        '主警', '缉毒', '组', '眼线', '贪污', '包', '毒品', '遭恶警', '加里', '奥德曼', '饰',
        '杀害', '全家', '惩罚', '马蒂达', '里昂', '留救', '幸免', '难留', '里昂', '里', '里昂',
        '教', '女孩', '枪教', '里昂', '法文', '两', '关系', '日趋', '亲密', '相处', '融洽'],
       dtype='<U4'),
 array(['年月日', '号称', '世界', '工业', '史迹', '豪华', '客轮', '泰坦尼克号', '处女航', '英国',
        '南安普顿', '发驶', '美国纽约', '富家', '少女', '罗丝', '凯特', '•', '温丝', '莱特',
        '母亲', '未婚夫', '卡坐', '头舱', '放荡', '羁', '少年', '画家', '杰', '克莱昂', '纳迪',
        '卡', '普里', '奥', '码头', '场', '赌博', '中', '赢', '舱', '船票'], dtype='<U5'))

# 欧式距离 （相似度越大，距离越小）

In [9]:
def count_boolean(strings,words_dict):
    words_vector = np.array([word for word in words_dict])
    return np.array([words_vector==s for s in strings]).sum(0)

In [10]:
s1 ,s2 = count_boolean(src1,words_dict),count_boolean(src2,words_dict)

In [11]:
def euclidean(s1,s2):
    return np.sqrt(np.power(s1-s2,2).sum())

In [12]:
euclidean(s1,s2)

2.23606797749979

# 雅卡尔系数 （不需要相同编码向量，直接可以比较,相似度越大，值越大）

In [13]:
def jaccard(s1,s2):
    return np.intersect1d(s1,s2).shape[0]/np.union1d(s1,s2).shape[0]

In [14]:
jaccard(src1,src1),jaccard(src1,src2)

(1.0, 0.0)

# 余弦相似度 （相似度越大，值越大）

In [15]:
def cosine(s1,s2):
    return s1.dot(s2)/(np.linalg.norm(s1) * np.linalg.norm(s2))

In [16]:
cosine(s1,s2),cosine(s1,s1)

(0.0, 1.0000000000000002)

3

# 曼哈顿距离 (相似度越大，值越小) 

In [17]:
def manhattan(s1, s2):
    n = min(s1.size,s2.size)
    return np.abs(s1-s2).dot(np.ones(n))

In [18]:
manhattan(s1,s2),manhattan(s1,s1)

(5.0, 0.0)

# 切比雪夫距离 (相似度越高,值越大,寻找的是文本当中的词最大的频次差异)

In [19]:
def chebyshev(s1, s2):
    return np.max(np.abs(s1 - s2))

In [20]:
chebyshev(s1,s2),chebyshev(s1,s1)

(1, 0)

# 闵可夫斯基距离,闵可夫斯基空间不同于牛顿力学的平坦空间。

In [21]:
def minkowski(x, y, p):
    return np.sum(np.abs(x - y) ** p) ** (1 / p)

In [22]:
minkowski(s1,s2,3),minkowski(s1,s1,7)

(1.7099759466766968, 0.0)

# 汉明距离 (统计的是相同位置里相同的数比例)

In [23]:
def hamming(x, y):
    return sum(x!=y)/len(x)

In [24]:
hamming(s1,s2)

3.191462200321699e-05

In [25]:
import numpy as np
y = 3
n = -5
e = -3
import itertools
S1 = np.array([y,n,e])
S = np.array(list(itertools.permutations(S1,3)))
S1+S,S1,S

(array([[  6, -10,  -6],
        [  6,  -8,  -8],
        [ -2,  -2,  -6],
        [ -2,  -8,   0],
        [  0,  -2,  -8],
        [  0, -10,   0]]),
 array([ 3, -5, -3]),
 array([[ 3, -5, -3],
        [ 3, -3, -5],
        [-5,  3, -3],
        [-5, -3,  3],
        [-3,  3, -5],
        [-3, -5,  3]]))

In [26]:
start = np.zeros_like(S)
start

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])